In [31]:
import os
import datetime
import torch
import random
import pickle
import string
import IPython
import numpy as np
import pandas as pd

#os.chdir(os.path.join(os.getcwd(), 'LAS Model'))
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from data import SpeechDataset, AudioDataLoader
from listener import Listener
from attend_and_spell import AttendAndSpell
from seq2seq import Seq2Seq
from utils import  train

### Load Training data

In [32]:
def preprocess(s):
    s = s.lower().replace('\n', '')
    return s.translate(str.maketrans('', '', string.punctuation)) # remove punctuation

# Used when each sentence is in a separate text file
def make_train_df(root_dir, dataset, file_ext, csv_file):
    dataset_dir = os.path.join(root_dir, dataset)
    data = []
    files = os.listdir(dataset_dir)
    for f in files:
        if '.txt' in f:
            with open(os.path.join(dataset_dir, f), 'r') as text_file:
                data_list = text_file.readlines()
            for example in data_list:
                path = os.path.join(dataset, str(example.split(' ')[0])) + file_ext   
                sent = preprocess(str(' '.join(example.split(' ')[1:])))
                data.append((path, sent))

    data_df = pd.DataFrame(data, columns=['path', 'sent'])
    data_df.to_csv(os.path.join(root_dir, csv_file), header=None)
    return data_df


root_dir = '../../../Dataset/LibriSpeech'
DEVICE = torch.device('cuda:1') if torch.cuda.is_available() else 'cpu'
print('DEVICE :', DEVICE)

DEVICE : cuda:1


In [33]:
# Create a train_df for each 100h, 360h, 500h datsets
make_train_df(root_dir, 'dataset_100', '.flac', 'train_100.csv') # for 100h
# read csv file
train_100 = pd.read_csv(os.path.join(root_dir, 'train_100.csv'), header=None, names=['path', 'sent'])
train_100.head()

,path,sent
0,dataset_100/103-1240-0000.flac,chapter one missus rachel lynde is surprised m...
1,dataset_100/103-1240-0001.flac,that had its source away back in the woods of ...
2,dataset_100/103-1240-0002.flac,for not even a brook could run past missus rac...
3,dataset_100/103-1240-0003.flac,and that if she noticed anything odd or out of...
4,dataset_100/103-1240-0004.flac,but missus rachel lynde was one of those capab...


In [48]:
make_train_df(root_dir, 'dataset_360', '.flac', 'train_360.csv') # for 360h
# Read csv file
train_360 = pd.read_csv(os.path.join(root_dir, 'train_360.csv'), header=None, names=['path', 'sent'])
train_360.head()

,path,sent
0,dataset_360/100-121669-0000.flac,tom the pipers son
1,dataset_360/100-121669-0001.flac,the pig was eat and tom was beat and tom ran c...
2,dataset_360/100-121669-0002.flac,he never did any work except to play the pipes...
3,dataset_360/100-121669-0003.flac,but he was so sly and cautious that no one had...
4,dataset_360/100-121669-0004.flac,and they lived all alone in a little hut away ...


In [35]:
make_train_df(root_dir, 'dataset_500', '.flac', 'train_500.csv') # for 500h
# read csv
train_500 = pd.read_csv(os.path.join(root_dir, 'train_500.csv'), header=None, names=['path', 'sent'])
train_500.head()

,path,sent
0,dataset_500/1006-135212-0000.flac,coming as it did at a period of exceptional du...
1,dataset_500/1006-135212-0001.flac,interest drooped however when after weeks of f...
2,dataset_500/1006-135212-0002.flac,it would be as well perhaps that i should refr...
3,dataset_500/1006-135212-0003.flac,in the year already mentioned a train left eus...
4,dataset_500/1006-135212-0004.flac,the train however is a favourite one among man...


In [49]:
# Combine all the datasets
train_df = pd.concat([train_100, train_360, train_500])
print("Number of training examples:", train_df.shape[0])

Number of training examples: 281241


### Creating Dictionary

In [46]:
def create_vocab(train_df):
    vocab = []
    for idx in range(train_df.shape[0]):
        _, sent = train_df.iloc[idx]
        words = sent.split(" ")
        for w in words:
            if w not in vocab:
                vocab.append(w)
        if idx==10000:
            break
    return vocab

vocab = create_vocab(train_df)
print("num words in vocab:", len(vocab))

num words in vocab: 20167


In [47]:
vocab

['chapter',
 'one',
 'missus',
 'rachel',
 'lynde',
 'is',
 'surprised',
 'lived',
 'just',
 'where',
 'the',
 'avonlea',
 'main',
 'road',
 'dipped',
 'down',
 'into',
 'a',
 'little',
 'hollow',
 'fringed',
 'with',
 'alders',
 'and',
 'ladies',
 'eardrops',
 'traversed',
 'by',
 'brook',
 'that',
 'had',
 'its',
 'source',
 'away',
 'back',
 'in',
 'woods',
 'of',
 'old',
 'cuthbert',
 'place',
 'it',
 'was',
 'reputed',
 'to',
 'be',
 'an',
 'intricate',
 'headlong',
 'earlier',
 'course',
 'through',
 'those',
 'dark',
 'secrets',
 'pool',
 'cascade',
 'but',
 'time',
 'reached',
 'lyndes',
 'quiet',
 'well',
 'conducted',
 'stream',
 'for',
 'not',
 'even',
 'could',
 'run',
 'past',
 'door',
 'without',
 'due',
 'regard',
 'decency',
 'decorum',
 'probably',
 'conscious',
 'sitting',
 'at',
 'her',
 'window',
 'keeping',
 'sharp',
 'eye',
 'on',
 'everything',
 'passed',
 'from',
 'brooks',
 'children',
 'up',
 'if',
 'she',
 'noticed',
 'anything',
 'odd',
 'or',
 'out',
 'woul

In [39]:

a = [1,2,3]
b = [4,5,6]
a.append(b)
a

[1, 2, 3, [4, 5, 6]]

### Load

In [3]:
#train_df = make_train_df(dataset_dir)
train_df = pd.read_csv(os.path.join(root_dir, 'train_df.csv'), names=['id', 'sent'])
train_df = train_df.dropna(how='any')
print("Num training examples", train_df.shape[0])
print(train_df.head())

Number of chars 32
Num training examples 93314
                id                                               sent
0  100-121669-0000                              TOM THE PIPER'S SON\n
1  100-121669-0001  THE PIG WAS EAT AND TOM WAS BEAT AND TOM RAN C...
2  100-121669-0002  HE NEVER DID ANY WORK EXCEPT TO PLAY THE PIPES...
3  100-121669-0003  BUT HE WAS SO SLY AND CAUTIOUS THAT NO ONE HAD...
4  100-121669-0004  AND THEY LIVED ALL ALONE IN A LITTLE HUT AWAY ...


In [4]:
tensorboard_dir = os.path.join('tb_summary')
train_dataset = SpeechDataset(train_df, dataset_dir, char_to_token, file_ext='.flac')
train_loader = AudioDataLoader(pad_token, train_dataset, batch_size=64, shuffle=True, drop_last=True)

### Instantiate model

In [5]:
load = False

if load:
    saved_file = 'Trained Models/Training_2019-12-25 00:09:23.921978/las_model_6'
    model.load_state_dict(torch.load(saved_file))
    start_epoch = int(saved_file[-1]) + 1
    time = os.listdir(tensorboard_dir)[-1]  # use the last one
else:
    start_epoch = 0
    time = str(datetime.datetime.now())

save_dir = os.path.join('trained_models_librispeech', f'Training_{time}')
try:    
    os.mkdir(save_dir);
except FileExistsError:
    pass

In [6]:
input_size = 128    # num rows in instagram
hidden_dim = 256  # 256*2 nodes in each LSTM
num_layers = 3
dropout = 0.1
layer_norm = False   
encoder = Listener(input_size, hidden_dim, num_layers, dropout=dropout, layer_norm=layer_norm)

hid_sz = 256
embed_dim = 30
vocab_size = len(chars)
decoder = AttendAndSpell(embed_dim, hid_sz, encoder.output_size, vocab_size)

hyperparams = {'input_size':input_size, 'hidden_dim':hidden_dim, 
               'num_layers':num_layers,'dropout':dropout, 
               'layer_norm':layer_norm, 'hid_sz':hid_sz, 
               'embed_dim':embed_dim, 'vocab_size':vocab_size}


# Saving hyperparmas
with open(os.path.join(save_dir, 'info.pickle'), 'wb') as f:
    pickle.dump(hyperparams, f)

criterion = nn.CrossEntropyLoss()
model = Seq2Seq(encoder, decoder, criterion, tf_ratio = 1.0, device=DEVICE).to(DEVICE)

### Training

In [ ]:
# optimizer = optim.ASGD(model.parameters(), lr=0.2)  # lr = 0.2 used in paper
optimizer = optim.Adadelta(model.parameters(), )
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.98)
log_interval = 5
print_interval = 40

epochs = 20
load = False

summary_dir = os.path.join(tensorboard_dir, time)
writer = SummaryWriter(summary_dir)


for epoch in range(start_epoch, epochs):
    print("\nTeacher forcing ratio:", model.tf_ratio)
    train(model, DEVICE, train_loader, optimizer, epoch, print_interval, writer, log_interval)
    scheduler.step()                                    # Decrease learning rate
    torch.save(model.state_dict(), os.path.join(save_dir, f'las_model_{epoch}'))
    model.tf_ratio = max(model.tf_ratio - 0.05, 0.8)    # Decrease teacher force ratio


Teacher forcing ratio: 1.0
Training, Logging: Mean loss of previous 40 batches 

Train Epoch: 0 [2496/93314 (3%)]	Mean Loss : 13.302426	 time 0:01:34.288800:
Train Epoch: 0 [5056/93314 (5%)]	Mean Loss : 13.108905	 time 0:01:32.994594:
Train Epoch: 0 [7616/93314 (8%)]	Mean Loss : 12.869395	 time 0:01:31.861588:
Train Epoch: 0 [10176/93314 (11%)]	Mean Loss : 12.590679	 time 0:01:29.919430:
Train Epoch: 0 [12736/93314 (14%)]	Mean Loss : 12.200976	 time 0:01:31.825354:
Train Epoch: 0 [15296/93314 (16%)]	Mean Loss : 11.901207	 time 0:01:31.531493:
Train Epoch: 0 [17856/93314 (19%)]	Mean Loss : 11.816909	 time 0:01:31.345978:
Train Epoch: 0 [20416/93314 (22%)]	Mean Loss : 11.709612	 time 0:01:34.322289:
Train Epoch: 0 [22976/93314 (25%)]	Mean Loss : 11.673304	 time 0:01:34.533173:
Train Epoch: 0 [25536/93314 (27%)]	Mean Loss : 11.666648	 time 0:01:34.239619:
Train Epoch: 0 [28096/93314 (30%)]	Mean Loss : 11.583640	 time 0:01:29.957466:
Train Epoch: 0 [30656/93314 (33%)]	Mean Loss : 11.58991

Train Epoch: 2 [76736/93314 (82%)]	Mean Loss : 11.271579	 time 0:01:36.648754:
Train Epoch: 2 [79296/93314 (85%)]	Mean Loss : 11.251705	 time 0:01:34.746429:
Train Epoch: 2 [81856/93314 (88%)]	Mean Loss : 11.287656	 time 0:01:30.358439:
Train Epoch: 2 [84416/93314 (90%)]	Mean Loss : 11.274141	 time 0:01:37.461059:
Train Epoch: 2 [86976/93314 (93%)]	Mean Loss : 11.246233	 time 0:01:34.373304:
Train Epoch: 2 [89536/93314 (96%)]	Mean Loss : 11.256752	 time 0:01:27.929825:
Train Epoch: 2 [92096/93314 (99%)]	Mean Loss : 11.254165	 time 0:01:28.820860:

Teacher forcing ratio: 0.8499999999999999
Training, Logging: Mean loss of previous 40 batches 

Train Epoch: 3 [2496/93314 (3%)]	Mean Loss : 11.359455	 time 0:01:30.132443:
Train Epoch: 3 [5056/93314 (5%)]	Mean Loss : 11.314065	 time 0:01:32.834080:
Train Epoch: 3 [7616/93314 (8%)]	Mean Loss : 11.327893	 time 0:01:32.992677:
Train Epoch: 3 [10176/93314 (11%)]	Mean Loss : 11.333943	 time 0:01:32.387827:
Train Epoch: 3 [12736/93314 (14%)]	Mean 

### TEST

In [6]:
def decode_pred_sent(out):
    pred_sent = []
    for t in out:
        lol = t.max(dim=1)[1].item()
        pred_sent.append(token_to_char[lol])
    return ''.join(pred_sent)


def decode_true_sent(y):
    sent = []
    for t in y:
        sent.append(token_to_char[t.item()])
    return ''.join(sent)

In [9]:
DEVICE = torch.device('cpu')

In [13]:
num_sent = 10
model.eval()
model.to(DEVICE)
model.device = DEVICE
model.tf_ratio = 0.9

for _ in range(num_sent):
    
    idx = random.randint(0, train_df.shape[0])
    trial_dataset = SpeechDataset(train_df, dataset_dir, sos_token, char_to_token, eos_token, file_extension='.flac')

    x, y = trial_dataset.__getitem__(idx)
    # plt.imshow(x[0,:,:].detach())

    # Model output
    target = y.unsqueeze(dim=0).to(DEVICE)
    data = x.permute(0, 2, 1).to(DEVICE)
    loss, output = model(data, target)
    print("True sent : ", decode_true_sent(y), end='\n\n')
    print("Pred sent : ", decode_pred_sent(output))
    print("Loss :", loss.item())    
    print("\n")

True sent :  <sos>who was now approaching womanhood he would sometimes talk with her differently from the manner in which he would speak to a mere girl but on her part she seemed not to notice the difference and for their daily amusement either go<eos>

Pred sent :   uholshs sornsnpropsh ng shmpndsun sorshuld shmp hnpl shll shlh hor shgfordds f soom shv sord r sn sholl sorshmld shrrssh nsnd r dsorlpsurosf sor srrshsho shnnud sor shvsor sorshvososfordd krsnd sor shv r sonnh snosh ond sngh<eos>r soo
Loss : 722.2744140625


True sent :  <sos>now sworn to the service of his most christian majesty<eos>

Pred sent :   uor shordssh shj shnoongdshosos sors soooshgnn soruss  
Loss : 178.7652587890625


True sent :  <sos>and a paper cap on his head has the strong conscience and the strong sense the blended susceptibility and self command of our friend adam he was not an average man yet such men as he are reared here and there in every generation of our peasant artisans<eos>

Pred sent :   und sh